# **Proyecto Final Unidad 6 Metodologias Agiles para Proyectos de Machine Learning**
---
##**Presentado por**:  
# Daniel Cardozo

##**Tema**:
# **Aplicación de Deep Learning para la Interpretacion Clínica de Pruebas de Sensibilidad Antimicrobiana (SIR)**



# **5.Despliegue de Modelo**
---

In [ ]:
!pip install mlflow==2.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, mlflow
from mlflow.tracking import MlflowClient


In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        --host 0.0.0.0 \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [ ]:
!pip install pyngrok

In [ ]:
token = "2zIRZxgk5ktWUn4mxcDYvZazqpL_5GDXjaS4WMRGmh6zPz9dY" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

<NgrokTunnel: "https://645518c25f8d.ngrok-free.app" -> "http://localhost:5000">

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Creamos un experimento:

In [ ]:
try:
  exp = mlflow.create_experiment(name="Clasificacion_SIR", artifact_location="mlruns/")
except:
  print("Revise que el código sea correcto, el experimento únicamente se puede crear una vez (evite correr el código varias veces)")

In [ ]:
# Para cada modelo en el diccionario
for nombre, df in X_train_list.items():
    antibiotico = nombre.replace("whonet_", "")
    df['LOCAL_ORG'] = pd.to_numeric(df['LOCAL_ORG'], errors='coerce')
    df[antibiotico] = df[antibiotico].astype('float32')

    y = y_train_list[nombre]
    weights = class_weights_list[nombre]

    with mlflow.start_run(
        run_name=f"modelo_{antibiotico}",
        experiment_id= exp) as run:

        mlflow.set_tag("antibiotico", antibiotico)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("batch_size", batch_size)

        model = build_single_model(df, n_classes=3)
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(
            x={'LOCAL_ORG': df['LOCAL_ORG'], 'atb': df[antibiotico]},
            y=y,
            epochs=100,
            batch_size=batch_size,
            callbacks=[checkpoint, stopping],
            class_weight=weights,
            validation_split=0.2,
            verbose=0
        )

        # Log automático del modelo Keras
        mlflow.keras.log_model(model, artifact_path="model")

        # Registrar métricas del último epoch
        final_epoch = len(history.history['loss']) - 1
        mlflow.log_metric("loss", history.history['loss'][final_epoch])
        mlflow.log_metric("val_loss", history.history['val_loss'][final_epoch])
        mlflow.log_metric("accuracy", history.history['accuracy'][final_epoch])
        mlflow.log_metric("val_accuracy", history.history['val_accuracy'][final_epoch])
        model_uri = f"runs:/{run.info.run_id}/model"
        result = mlflow.register_model(model_uri=model_uri, name=f"modelo_{antibiotico}")

        # ✅ TRANSICIÓN A PRODUCCIÓN
        client = MlflowClient()
        client.transition_model_version_stage(
        name=f"modelo_{antibiotico}",
        version=result.version,  # <- aquí sí está definido
        stage="Production",
        archive_existing_versions=True)


2025/07/24 19:50:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/24 19:50:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'modelo_AMK_NM'.
2025/07/24 19:50:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_AMK_NM, version 1
Created version '1' of model 'modelo_AMK_NM'.
/tmp/ipython-input-41-4080764270.py:48: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
2025/

In [ ]:
#especificamos la url del servidor de seguimiento de mlflow
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"


In [ ]:
#Lanzamosel API con mlflow
command = """
mlflow models serve -m 'models:/modelo_amk_nm/Production' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)

In [ ]:
import requests
import json

In [ ]:
data_requesr= df['AMK_NM']

In [ ]:
r = requests.post(
    "http://localhost:8001/invocations", json = {"inputs": data_request})
print(r.text)
